# ProGym - New Faciliy near Phoenix, AZ

ProGym is a fictional physical fitness and training company based in Los Angeles, CA.  The ProGym Business Model caters primarily to physicians and members of the professional medical community.  The ProGym management team has been growing the company steadilly since 2014 and believes that Phoenix, AZ is the best location for the next facility.

DataHAX is a fictional Business Intellegence firm based in Tucson, AZ.  One of the core offerings by DataHAX is understanding business environments.  DataHAX mission is simple - we turn data into intelligence, and intelligence into success.  For almost 90 days we have been using our intelligence assets to help businesses win.


## Business Challenge
DataHAX has been hired by ProGym to help determine the best location for a new facility in the metro Phoenix area.  ProGym has a deep understanding of their highest value facilities and what makes them profitable and successful. From their most successful facilities, ProGym as developed a **Facility Profile**. ProGym understands their clientele very well and have developed a very specific **Client Profile** based on their business model.  The challenge for DataHAX will be to combine data about the Phoenix Metro Area with the ProGym **Client and Facility Profiles** to find the best location candidates.

## Stakeholders
Interested parties for the ProGym-Phoenix project include the ProGym facilities team and the ProGym executive management team.

## The Data
- FourSquare.com - Public venue information service:  https://developer.foursquare.com/docs/places-api/
- US Census - Age/Sex/Education by ZIPcode for Arizona: https://data.census.gov - tables: S0101, S1501 and B19001
- US Internal Revenue Service - Income Data by ZIPcode for Arizona: https://www.irs.gov/pub/irs-soi/17zp03az.xlsx
- Zillow - Residential Home values by ZIPcode: http://files.zillowstatic.com/research/public_v2/zhvi/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv

### ProGym Facility Profile
ProGym facilities located near hospitals, doctor's offices and other professional buildings have proven to be the most successful.  Additionally, ProGym has noted that locations in close proximity to fine dining establishments tend to sell more personal training services.  This correlation is strong enough to indicate that some weight should be given to proximity to fine dining.  
- From the FourSquare.com Dataset API, will extract the following for each ZIPcode (to allow an analysis of the ProGym Facility Profile):
  - Hospital Count
  - Doctors office Count
  - Fine Dining Restaurant Count

### ProGym Client Profile
ProGym clientle are primarily medical professionals.  This leads to the following demographics:  age, education level, income level, and others.

Each dataset for this exploration consists of records indexed by a 5-digit ZIPcode. 
- From the US Census Dataset, will extract the following for each ZIPcode:
  - Total age 35-55
  - Age 35-55 with Bachleors Degree
  - Age 35-55 with Bachleors Degree or higher
  - Age 35-55 Professional Degree 
- From the Internal Revenue Service Dataset, income level for each ZIPcode will be added to the dataset
- From Zillow Dataset, the current median home value will be added to the dataset

- Each record in the new dataset will look like the following (to allow an analysis of the ProGym client profile):
  - ZIP Code ( Index )
  - Total age 35-55
  - Age 35-55 with Bachelors Degree - count
  - Age 35-55 with Bachelors Degree or higher - count
  - Age 35-55 Professional Degree - count
  - Median Income
  - Median Home Value  

## Data Wrangling
The data for this evaluation come from several sources.  For analysis, each dataset needs to be cleaned up an normalized.  The following code produces Pandas DataFrames with a ZIP code index for each dataset.

In [1]:
import pandas as pd
import numpy as np
import folium
import requests
import pickle
import time

# All of our data will be relating to the postal codes (ZIP Codes) in Maricopa County (i.e. Phoenix Metro Area), AZ, USA
maricopa_zips = ["85004","85007","85545","85006","85009","85008","85011","85013","85012","85015","85014","85017","85016","85019","85018","85021","85020","85023","85022","85024","85027","85026","85029","85028","85031","85033","85032","85035","85034","85037","85286","85036","85083","85038","85190","85041","85040","85043","85042","85045","85044","85048","85392","85395","85295","85051","85298","85050","85054","85053","85064","85067","85085","85082","85087","85086","85098","85202","85201","85204","85203","85206","85205","85208","85207","85210","85209","85212","85213","85215","85219","85224","85226","85225","85233","85234","85236","85244","85249","85248","85251","85250","85253","85252","85255","85254","85257","85256","85259","85258","85261","85260","85263","85262","85266","85264","85268","85274","85282","85281","85284","85283","85287","85118","85297","85296","85301","85142","85303","85139","85302","85305","85304","85307","85306","85308","85310","85320","85322","85323","85326","85327","85331","85329","85333","85335","85337","85339","85338","85340","85343","85342","85345","85351","85353","85355","85354","85358","85361","85363","85373","85375","85374","85377","85379","85378","85381","85383","85382","85387","85385","85390","85388","85396","85001","85003","85002"]

## FourSquare API Setup

In [2]:
# Define and download the wiki page
CLIENT_ID = '5HPET1K2F5KSIMNIAOJSTRSQBWW3CO5YTVYI5UQDO0DACZLH' # your Foursquare ID
CLIENT_SECRET = 'MDZZBZNLQ0XDHUCY43AVPJDXXYPYOKYFZKT4SYZRFSONU1S5' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50

In [3]:
# A dict to hold values returned from the FourSquare API
venue_dict = {} # This will be populated with arrays for each of 'venue_cols' 
venue_cols = ["ZIP", "id", "Name", "Latitude", "Longitude", "Category", "Search", "Price Tier", "Rating", "Likes"]

# The FourSquare API returns many irrelivant categories - this is the sort list of categories in which we're interested
keep_med_cats  = ["Hospital","Dentist's Office","Doctor's Office","Emergency Room","Medical Center"]
keep_food_cats = ["Restaurant","Wine Bar"]

## Function to get and prep FourSquare Data
This function reads from the FourSqure API and prepares a dictionary.

The dictionary will be translated into a Pandas DataFrame later.

This routine finds medical facilities for each zip code and restaurants in the 3-4 (more expensive) price range.

In [4]:
# Cached data - so we can limit premium API calls
venue_dict = pickle.load(open("data/venues.pkl", 'rb'))

In [5]:
# This function takes a data dictionary, a list of zip codes, and a search query
# Using the FourSquare REST API, we'll assemble of list of venues 
# the dataset is defined by 'venue_cols' defined above.  
# Venues are stored in the dataset only if they match a category in 'keep_categories'
# This is soooo much easier with the FourSquare Python API: https://pypi.org/project/foursquare/
# 

def get_venues_by_zip(the_dict, zip_list, search_query):
    
    # Food and Medical have slightly different execution paths so, it's important
    # to distinguish between the two query types
    if search_query in ["Hospital", "Doctor's Office"]:
        query_type = 'Medical'
    elif search_query in ['Restaurant']:
        query_type = 'Food'
    else:
        raise QuerryError

    if the_dict == {}:  # Needs to be initialized!
        the_dict = {x:[] for x in venue_cols}
    
    radius = 20000
    LIMIT = 50
    premium_calls = 0
    duplicate_venue_hits = 0
    unique_venue_hits = 0

    for postal_code in zip_list:
        
        if query_type == 'Medical':
            # Set the medical facility search url
            url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&v={}&query={}&limit={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, postal_code, VERSION, search_query, LIMIT, radius)
        else:
            # Set the Restaurant/Price Query
            # FourSquare rates prices on a scale of 1-4, we're interested only in '3&4' level pricing
            url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={}&v={}&query={}&limit={}&radius={}&price=3,4'.format(CLIENT_ID, CLIENT_SECRET, postal_code, VERSION, search_query, LIMIT, radius)

        results = requests.get(url).json()
        
        if query_type == 'Medical':
            try:
                venues  = results['response']['venues'] 
            except KeyError:
                continue  # We've got nothing to work with go to the next zip!
        else:
            try:
                venues = []
                match_list = results['response']['groups'][0]['items']
                for match in match_list:
                    venues.append(match['venue'])
            except KeyError: # There are no venues that match this query, move along
                continue
        for venue in venues:    
            if venue['id'] in the_dict['id']:  # We have a duplicate venue
                duplicate_venue_hits += 1
                continue
            else:
                unique_venue_hits += 1
            
            # Skip venues that don't have a category or have the wrong category
            if len(venue['categories']) == 0 : continue
                
            # Have a category - let's see if it's one that we care about
            the_cat = venue['categories'][0]['name']
            
            # There's not much elegance to the catagory logic so, I just hammered it out in long-hand... 
            # We have medical venues and food venues - they are treated differently
            # So we'll set a flag for each
            
            # Medical Categories that are relevant
            med_keeper = (the_cat in keep_med_cats)
            
            # Food Catetories that are relevant
            food_keeper = ((the_cat.endswith('Restaurant')) or (the_cat == 'Wine Bar') or (the_cat == 'Food')) and (the_cat != 'Fast Food Restaurant')
                 
            if not ( med_keeper or food_keeper):
                continue # We don't have a relevant catagory - move along
                
            # We have a good category - add it to the list
            the_dict["Category"].append(the_cat) 
            
            zip_code = venue['location']['postalCode'] if 'postalCode' in venue['location'] else postal_code
            the_dict['ZIP'].append(zip_code)
            
            the_dict["id"].append(venue["id"])
            the_dict["Name"].append(venue["name"])
            the_dict["Latitude"].append(venue['location']['lat'])
            the_dict["Longitude"].append(venue['location']['lng'])
            
            price_tier   = np.nan
            venue_rating = np.nan
            venue_likes  = np.nan

            if food_keeper: # We need to get the rating and price range
                url = "https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}".format(venue["id"],CLIENT_ID, CLIENT_SECRET, VERSION)
                # Get a JSON response from FourSquare
                premium_calls += 1
                results = requests.get(url).json()
                
                venue = results['response']['venue']
                # This is a bit ugly - but only way to check if a 
                try:
                    price_tier = venue['price']['tier']
                except KeyError:
                    pass
                try:
                    venue_rating = venue['rating']
                except KeyError:
                    pass
                try:
                    venue_likes = venue['likes']['count']
                except KeyError:
                    pass   
                 
            the_dict["Price Tier"].append(price_tier)
            the_dict["Rating"].append(venue_rating)
            the_dict["Likes"].append(venue_likes)
            the_dict["Search"].append(search_query)
             
    print("Premium Calls:   {0}".format(premium_calls))
    print("Unique Venues:   {0}".format(unique_venue_hits))
    print("Duplicate Venus: {0}".format(duplicate_venue_hits))
    return the_dict


In [6]:
venue_dict = get_venues_by_zip(venue_dict, maricopa_zips, "Hospital")
venue_dict = get_venues_by_zip(venue_dict, maricopa_zips, "Doctor's Office")
venue_dict = get_venues_by_zip(venue_dict, maricopa_zips, "Restaurant")

Premium Calls:   0
Unique Venues:   4058
Duplicate Venus: 2529
Premium Calls:   1
Unique Venues:   6237
Duplicate Venus: 919
Premium Calls:   0
Unique Venues:   656
Duplicate Venus: 1628


In [11]:
# Persist the venues data set for later...
file = open('data/venues.pkl', 'wb')
# dump information to that file
pickle.dump(venue_dict, file)
# close the file
file.close()

In [12]:
medical_venues_df = pd.DataFrame(data=venue_dict,columns=venue_cols)

In [13]:
medical_venues_df

,ZIP,id,Name,Latitude,Longitude,Category,Search,Price Tier,Rating,Likes
0,85018,4ebb282fb803cf8e0e7e8871,North Italia,33.509048,-111.994964,Italian Restaurant,Restaurant,3.0,9.2,140.0
1,85018,5342cc6c498e6451000eb099,Steak 44,33.511357,-111.986222,Restaurant,Restaurant,3.0,9.0,105.0
2,85018,4b184790f964a52020d023e3,T. Cook's,33.505052,-111.968447,Mediterranean Restaurant,Restaurant,3.0,8.9,57.0
3,85018,4b25311ff964a520b26d24e3,Tarbell's,33.509227,-112.012379,American Restaurant,Restaurant,4.0,8.8,41.0
4,85016,4c8eafbd1664b1f76e76a22f,Seasons 52,33.510501,-112.026691,New American Restaurant,Restaurant,3.0,9.1,104.0
...,...,...,...,...,...,...,...,...,...,...
412,85251,4e6fab1dfa765666a302c61a,Dr. Johnny Walker's Office,33.488882,-111.915038,Doctor's Office,Doctor's Office,NaN,NaN,NaN
413,85251,4e8210845503a464d8ff2a92,Dr. Sibley's Office,33.489807,-111.923531,Doctor's Office,Doctor's Office,NaN,NaN,NaN
414,85251,4d530ad7bd6ff04d2d31020d,Dentist's Office,33.486942,-111.919638,Dentist's Office,Doctor's Office,NaN,NaN,NaN
415,85006,52d85d0e498e4bedb56bae19,Phoenix childrens hospital emergency,33.478653,-112.039848,Emergency Room,Hospital,NaN,NaN,NaN


medical_venues_df.shape => (415,10) - before first run
## Prepare US Census Tables
Tables from https://data.census.gov are over 300 columns - of which we require only 6.  In this section we'll clean up 

### Clean Up the Population By Age Table

In [14]:
# This table can be downloaded at
# https://data.census.gov/cedsci/table?q=All%20County%20Subdivisions%20within%20Arizona&hidePreview=true&tid=ACSST5Y2018.S0101&vintage=2018&layer=VT_2018_860_00_PY_D1&cid=S0101_C01_001E&g=8600000US85027,85023,85022,85306,85053,85310,85308,85382,85024,85050,85032,85254,85260,85054,85381,85351,85335,85363,85345,85304,85029,85051,85302,85021,85020,85028,85253,85251,85016,85014,85015,85013,85012,85017,85019,85301,85303,85305,85307,85309,85392,85035,85257,85018,85009,85031,85033,85037,85256,85250,85258,85034,85008,85006,85007,85003,85004,85043,85041,85353,85323,85338,85395,85204,85286,85248,85048,85226,85044,85042,85040,85282,85281,85205,85207,85140,85212,85296,85284,85283,85249,85203,85213,85215,85259,85201,85202,85224,85225,85233,85234,85206,85210,85209,85208,85295,85297,85298,85379,85375,85373,85383,85268,85255,85339,85355,85388,85374,85340,85387
population_by_age_df = pd.read_csv("data/ACSST5Y2018.S0101_data_with_overlays_2020-06-27T164439.csv")
population_by_age_df.shape

(110, 458)

### Drop columns that are not required

In [15]:
# These are the columns that are relevant for this investigation
keep_cols = ["Geographic Area Name","Estimate!!Total!!Total population!!AGE!!35 to 39 years","Estimate!!Total!!Total population!!AGE!!40 to 44 years","Estimate!!Total!!Total population!!AGE!!45 to 49 years","Estimate!!Total!!Total population!!AGE!!50 to 55 years"]

# Drop irrelevant column
population_by_age_df.drop([col for col in population_by_age_df.columns if col not in keep_cols],axis=1,inplace=True)

# ZIP Code is the last 5 digits of Geographic Area Name
population_by_age_df["Geographic Area Name"] = population_by_age_df["Geographic Area Name"].str[-5:] 

In [16]:
# The raw headers are kind of ugly - use this function to clean up (lots of) unnecessary characters
def clean_pop_by_age_col_names(the_name):
    the_name = the_name.replace("Estimate!!Total!!Total population!!AGE!!", "")
    the_name = the_name.replace("Geographic Area Name", "ZIP")
    the_name = the_name.rstrip(" ")
    the_name = the_name.lstrip(" ")
    
    return the_name.strip()

In [17]:
population_by_age_df.rename(columns={x:clean_pop_by_age_col_names(x) for x in population_by_age_df.columns}, inplace=True)
population_by_age_df.shape

(110, 4)

In [18]:
population_by_age_df.head(5)

,ZIP,35 to 39 years,40 to 44 years,45 to 49 years
0,85003,729,798,490
1,85004,303,284,338
2,85006,2218,1796,1350
3,85007,1083,801,769
4,85008,4108,3725,4147


### Clean up Population by age and Education Table

In [19]:
# This table may be downloaded at:
# https://data.census.gov/cedsci/table?q=All%20County%20Subdivisions%20within%20Arizona&hidePreview=true&tid=ACSST5Y2018.S1501&vintage=2018&layer=VT_2018_860_00_PY_D1&cid=S0101_C01_001E&g=8600000US85027,85023,85022,85306,85053,85310,85308,85382,85024,85050,85032,85254,85260,85054,85381,85351,85335,85363,85345,85304,85029,85051,85302,85021,85020,85028,85253,85251,85016,85014,85015,85013,85012,85017,85019,85301,85303,85305,85307,85309,85392,85035,85257,85018,85009,85031,85033,85037,85256,85250,85258,85034,85008,85006,85007,85003,85004,85043,85041,85353,85323,85338,85395,85204,85286,85248,85048,85226,85044,85042,85040,85282,85281,85205,85207,85140,85212,85296,85284,85283,85249,85203,85213,85215,85259,85201,85202,85224,85225,85233,85234,85206,85210,85209,85208,85295,85297,85298,85379,85375,85373,85383,85268,85255,85339,85355,85388,85374,85340,85387
population_by_edu_df = pd.read_csv("data/ACSST5Y2018.S1501_data_with_overlays_2020-06-27T233911.csv")

In [20]:
# Same as above, drop the irrelevant columns, and clean up the ugly names
keep_cols = ["Geographic Area Name","Estimate!!Total!!Population 25 years and over!!Population 35 to 44 years!!Bachelor's degree or higher","Estimate!!Total!!Population 25 years and over!!Population 45 to 64 years!!Bachelor's degree or higher","Estimate!!Total!!Population 25 years and over!!Graduate or professional degree"]

# Drop the irrelevant columns
population_by_edu_df.drop([col for col in population_by_edu_df.columns if col not in keep_cols],axis=1,inplace=True)

# ZIP Code is the last 5 digits of Geographic Area Name
population_by_edu_df["Geographic Area Name"] = population_by_edu_df["Geographic Area Name"].str[-5:] 

In [21]:
def clean_ed_cols(the_name):
    the_name = the_name.replace("!!", " ")
    the_name = the_name.replace("Estimate", "").strip()
    the_name = the_name.replace("Population 25 years and over Population", "")
    the_name = the_name.replace("Geographic Area Name","ZIP")
    the_name = the_name.replace("  ", " ")
    
    return the_name
    


In [22]:
# Use the function defined above to clean up the column names
population_by_edu_df.rename(columns={x:clean_ed_cols(x) for x in population_by_edu_df.columns}, inplace=True)
population_by_edu_df.head(2)

,ZIP,Total 35 to 44 years Bachelor's degree or higher,Total 45 to 64 years Bachelor's degree or higher,Total Population 25 years and over Graduate or professional degree
0,85003,670,851,1089
1,85004,201,536,580


### Clean-up the IRS/Income Tables

In [23]:
# This data is pretty clean - just pull it out of the xlsx
# Excel file can be found here:
income_by_zip_df = pd.read_excel("data/17zp03az.xlsx", sheet_name="Sheet2",skiprows=3, skipfooter=2,usecols="a:d",names=["ZIP", "75K to 100K", "100K to 200K", "Totals"])

In [24]:
income_by_zip_df.head()

,ZIP,75K to 100K,100K to 200K,Totals
0,85003,340,580,920
1,85004,300,450,750
2,85006,470,460,930
3,85007,280,400,680
4,85008,940,840,1780


## Prepare Zillow Data
### Clean Up Zillow Data

In [25]:
# The Zillow data may be downloaded here:
# https://www.zillow.com/research/data/
home_values_df = pd.read_csv("data/zillow_data.csv")

In [26]:
# Rename to fit our index scheme
home_values_df.rename(columns={"RegionName":"ZIP", "2020-01-31":"Mean Value - Jan 2020"},inplace=True)

In [27]:
# Drop all of the irrelevant values
home_values_df = home_values_df.loc[home_values_df['ZIP'].isin(maricopa_zips)]

In [28]:
# Same as above, drop the irrelevant columns, and clean up the ugly names
keep_cols = ['ZIP','Mean Value - Jan 2020']

# Drop the irrelevant columns
home_values_df.drop([col for col in home_values_df.columns if col not in keep_cols],axis=1,inplace=True)

In [29]:
home_values_df.head()

,ZIP,Mean Value - Jan 2020
71,85032,287971.0
107,85281,281139.0
116,85225,284867.0
148,85308,309422.0
300,85142,350664.0
